# Data Import

In [1]:
import pandas as pd
items_dataset = pd.read_csv('D:/Uni Mannheim/Semester 2/Data Mining 2/DMC/items.csv', sep='|')
items_dataset.head(1)

,itemID,title,author,publisher,main topic,subtopics
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH]


In [2]:
import json
# Opening JSON file
f = open('D:/Uni Mannheim/Semester 2/Data Mining 2/DMC/gbooks_final.json',)  
# returns JSON object as 
# a dictionary
data = json.load(f)  
# Closing file
f.close()

# Data Preprocessing

In [3]:
items_google = data.get('items')
totalItems_google = data.get('totalItems')

In [4]:
df_external = pd.DataFrame(columns = ['itemID', 'language', 'description', 'ISBN_13'])

In [6]:
#Data is extracted from a nested json file to a pandas dataframe! :D
for i in range (len(items_dataset.index)):
    key = list(items_google)[i]
    total_items = list(totalItems_google.values())[i]
    if (total_items == None):
        total_items = 0
    items=list(items_google.values())[i]
    current_id = current_id =  items_dataset.iloc[i].at['itemID']
    current_description = ''
    current_language = ''
    current_ISBN = ''
    if (total_items == 1) :
        only_item = items[0]
        current_volume = only_item.get('volumeInfo')
        current_description = current_volume.get('description')
        current_language = current_volume.get('language')
        current_industryIdentifiers = current_volume.get('industryIdentifiers')
    elif (total_items > 1):
        for item in items:
            counter = 0
            current_volume = item.get('volumeInfo')
            current_publisher = current_volume.get('publisher')
            if (current_publisher == items_dataset.iloc[i].at['publisher']):
                counter = counter + 1
                current_description = current_volume.get('description')
                current_language = current_volume.get('language')
                current_industryIdentifiers = current_volume.get('industryIdentifiers')
        if  (current_description == '' ):
            random_item = items[0]
            current_volume = random_item.get('volumeInfo')
            current_description = current_volume.get('description')
            current_language = current_volume.get('language')
            current_industryIdentifiers = current_volume.get('industryIdentifiers')
    df_external.at[i,'itemID'] = current_id
    df_external.at[i,'language'] = current_language
    df_external.at[i,'description'] = current_description
    df_external.at[i,'ISBN_13'] = current_industryIdentifiers
df_external.head(20)    

,itemID,language,description,ISBN_13
0,21310,en,A brand-new story in the best-selling Princess...,"[{'type': 'ISBN_13', 'identifier': '9781409048..."
1,73018,de,None,"[{'type': 'ISBN_10', 'identifier': '3849927601..."
2,19194,en,The first novel in the #1 bestselling RED QUEE...,"[{'type': 'ISBN_13', 'identifier': '9781409150..."
3,40250,de,Ein wunderschönes Freundebuch zur Erinnerung a...,"[{'type': 'ISBN_13', 'identifier': '9798691079..."
4,46107,,,"[{'type': 'ISBN_13', 'identifier': '9798691079..."
5,34217,de,Rhage und Mary sind Geliebte und Seelenverwand...,"[{'type': 'ISBN_13', 'identifier': '9783641195..."
6,31436,de,None,"[{'type': 'ISBN_10', 'identifier': '3845817445..."
7,14576,de,Nur sie kann die Menschheit retten Unsere Welt...,"[{'type': 'ISBN_13', 'identifier': '9783641100..."
8,17731,de,Nur sie kann die Menschheit retten Unsere Welt...,"[{'type': 'ISBN_10', 'identifier': '3453268571..."
9,58723,de,Kind; Armut; Dritte Welt; Kinderbuch; Kolumbien.,"[{'type': 'ISBN_10', 'identifier': '3423702486..."


In [35]:
#now we just join the result with our original dataset
items_annotated_temp = items_dataset.merge(df_external, how='left', on='itemID')

In [36]:
#Here, I just imported item_with_age_identifier.csv provided by Chihyen
item_with_age_identifier = pd.read_csv('D:/Uni Mannheim/Semester 2/Data Mining 2/DMC/item_with_age_identifier.csv', sep=',')

In [37]:
item_with_age_identifier.head(1)

,Unnamed: 0,itemID,title,author,publisher,main topic,subtopics,interest_age
0,0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,['5AH'],1


In [38]:
item_with_age_identifier.drop(['Unnamed: 0','title', 'author', 'publisher','main topic', 'subtopics'],
  axis= 1, inplace=True)

In [39]:
item_with_age_identifier.head(1)

,itemID,interest_age
0,21310,1


In [40]:
#and finally join it to the final result
items_annotated = items_annotated_temp.merge(item_with_age_identifier, how='left', on='itemID')

In [41]:
items_annotated.head()

,itemID,title,author,publisher,main topic,subtopics,language,description,ISBN_13,interest_age
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH],en,A brand-new story in the best-selling Princess...,"[{'type': 'ISBN_13', 'identifier': '9781409048...",1
1,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]",de,None,"[{'type': 'ISBN_10', 'identifier': '3849927601...",1
2,19194,Red Queen 1,Victoria Aveyard,Orion Publishing Group,YFH,"[5AP,FBA]",en,The first novel in the #1 bestselling RED QUEE...,"[{'type': 'ISBN_13', 'identifier': '9781409150...",1.5
3,40250,Meine Kindergarten-Freunde (Pirat),NaN,Ars Edition GmbH,YB,"[5AC,5AD,YBG,YBL,YF]",de,Ein wunderschönes Freundebuch zur Erinnerung a...,"[{'type': 'ISBN_13', 'identifier': '9798691079...",0
4,46107,Mein großes Schablonen-Buch - Wilde Tiere,Elizabeth Golding,Edition Michael Fischer,WFTM,"[WD,WFTM,YBG,YBL,YBLD,YBLN1]",,,"[{'type': 'ISBN_13', 'identifier': '9798691079...",0


In [42]:
items_annotated.to_csv('annotated_item_dataset.csv')

In [46]:
# Now we use langDetect to improve the result: it can help us to replace 6100 empty values for language attribute by its suggestions.
items_annotated ['LangDetect'] = ''
!pip install langdetect
from langdetect import DetectorFactory, detect
from langdetect.lang_detect_exception import LangDetectException
DetectorFactory.seed = 0
for i in range (len(items_annotated.index)):
    concat_string = str(items_annotated.loc[i].at['title']) + str(items_annotated.loc[i].at['author']) + str(items_annotated.loc[i].at['publisher']) + str(items_annotated.loc[i].at['description'])
    try:
        language = detect(concat_string)
    except LangDetectException:
        language = 'unknown'
    items_annotated.at[i,'LangDetect'] = language

In [64]:
items_annotated.head()

,itemID,title,author,publisher,main topic,subtopics,language,description,ISBN_13,interest_age,LangDetect
0,21310,Princess Poppy: The Big Mix Up,Janey Louise Jones,Penguin Random House Children's UK,YFB,[5AH],en,A brand-new story in the best-selling Princess...,"[{'type': 'ISBN_13', 'identifier': '9781409048...",1,en
1,73018,Einfach zeichnen! Step by Step,Wiebke Krabbe,Schwager und Steinlein,AGZ,"[5AJ,AGZ,WFA,YBG,YBL,YNA,YPA]",de,None,"[{'type': 'ISBN_10', 'identifier': '3849927601...",1,de
2,19194,Red Queen 1,Victoria Aveyard,Orion Publishing Group,YFH,"[5AP,FBA]",en,The first novel in the #1 bestselling RED QUEE...,"[{'type': 'ISBN_13', 'identifier': '9781409150...",1.5,en
3,40250,Meine Kindergarten-Freunde (Pirat),NaN,Ars Edition GmbH,YB,"[5AC,5AD,YBG,YBL,YF]",de,Ein wunderschönes Freundebuch zur Erinnerung a...,"[{'type': 'ISBN_13', 'identifier': '9798691079...",0,de
4,46107,Mein großes Schablonen-Buch - Wilde Tiere,Elizabeth Golding,Edition Michael Fischer,WFTM,"[WD,WFTM,YBG,YBL,YBLD,YBLN1]",de,,"[{'type': 'ISBN_13', 'identifier': '9798691079...",0,de


In [62]:
for i in range (len(items_annotated.index)):
    if ((pd.isnull(items_annotated.loc[i, 'language'])) or (items_annotated.loc[i, 'language'] == None) or (items_annotated.loc[i, 'language'] == '')):
        lang_temp = items_annotated.loc[i, 'LangDetect']
        items_annotated.loc[i, 'language'] = lang_temp            

In [65]:
disagree = 0
for i in range (len(items_annotated.index)):
    if (items_annotated.loc[i, 'language'] != items_annotated.loc[i, 'LangDetect']):
        disagree = disagree + 1

In [66]:
print (disagree)

4278


In [67]:
items_annotated.to_csv('annotated_item_dataset.csv')